In [1]:
%autosave 60
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
from pathlib import Path

Autosaving every 60 seconds


In [2]:
p = Path("../").resolve()

In [3]:
import json
import os
import pickle
from collections import Counter
from copy import deepcopy
from io import BytesIO
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union, cast

import cv2
import matplotlib as plt
import numpy as np
import pandas as pd
import PIL
import PIL.Image as pil_img
import seaborn as sns
import sklearn as skl
from geoscreens.consts import GEO_SCREENS, IMG_SIZE
from geoscreens.geo_data import GeoScreensDataModule
from geoscreens.modules import LightModelTorch, build_module
from icevision import models, tfms
from icevision.all import *
from icevision.data import Dataset, DataSplitter, RandomSplitter
from icevision.parsers.coco_parser import COCOBBoxParser
from IPython.display import Image, display
from matplotlib.patches import Rectangle
from matplotlib_inline.backend_inline import set_matplotlib_formats
from pytorch_lightning import LightningDataModule, Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger
from scripts.train_geo import get_model
from tqdm.notebook import tqdm

INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /home/gbiamby/.icevision/mmdetection_configs/mmdetection_configs-2.16.0/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17


In [4]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 15)
pd.set_option("display.max_rows", 50)
# Suitable default display for floats
pd.options.display.float_format = "{:,.2f}".format
plt.rcParams["figure.figsize"] = (12, 10)

# This one is optional -- change graphs to SVG only use if you don't have a
# lot of points/lines in your graphs. Can also just use ['retina'] if you
# don't want SVG.
%config InlineBackend.figure_formats = ["retina"]
set_matplotlib_formats("pdf", "png")

In [5]:
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("pdf", "png")
plt.rcParams["savefig.dpi"] = 75

plt.rcParams["figure.autolayout"] = False
plt.rcParams["figure.figsize"] = 10, 6
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["font.size"] = 16
plt.rcParams["lines.linewidth"] = 2.0
plt.rcParams["lines.markersize"] = 8
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["text.usetex"] = True

plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "cm"
plt.rcParams["text.latex.preamble"] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

<ipython-input-5-55fc3ad33997>:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("pdf", "png")


In [6]:
VIDEO_PATH = Path("/shared/g-luo/geoguessr/videos").resolve()
OUT_PATH = Path("/shared/gbiamby/geo/segment").resolve()
assert VIDEO_PATH.exists()
assert OUT_PATH.exists()

files = sorted(VIDEO_PATH.glob("**/*.mp4"))
print("total video files found: ", len(files))

total video files found:  4084


In [7]:
# def sample_frames(video_path: Path, out_path: Path, sample_every_seconds=20.0):
#     """
#     Sample a frame every `sample_every_seconds` seconds from the specified video, saving it to
#     """
#     path = video_path.with_suffix(".mp4")
#     assert path.exists(), str(path)
#     cap = cv2.VideoCapture(str(path))
#     if not cap.isOpened():
#         print("could not open :", video_path)
#     num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print(f"num_frames: {num_frames:,}")
#     seconds = round(0, 2)
#     sample_every_seconds = 5.0
#     count = 0
#     success = True
#     print(path)
#     while success:
#         cap.set(cv2.CAP_PROP_POS_MSEC, (seconds * 1000))
#         success, image = cap.read()
#         if success:
#             frame_out_path = out_path / "screen_samples_auto" / f"{path.stem}/frame_{count:08}.jpg"
#             frame_out_path.parent.mkdir(exist_ok=True, parents=True)
#             # print(frame_out_path)
#             cv2.imwrite(str(frame_out_path), image)
#         seconds = round(seconds + sample_every_seconds, 2)
#         count += 1
#     cap.release()
#     print("total frames captured: ", count, ", seconds: ", seconds)


# for i, (idx, row) in tqdm(enumerate(df_sampled.iterrows()), total=len(df_sampled)):
#     # if i > 10:
#     #     break
#     sample_frames(row.path, OUT_PATH, 20.0)

## Load Detection Model

In [8]:
DEVICE = torch.device("cuda:2")
seed_everything(42, workers=True)
geo_screens = GeoScreensDataModule()

Global seed set to 42


  0%|          | 0/3326 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/888 [00:00<?, ?it/s]

AUTOFIX-SUCCESS - (record_id: 712) - Clipping bbox ymin from -2 to 0 (Before: (<BBox (xmin:908, ymin:-2, xmax:1280, ymax:35)>)) | icevision.utils.logger_utils:autofix_log:17


classes:  <ClassMap: {'background': 0, 'between_rounds_box_white': 1, 'between_rounds_box_with_orange_next': 2, 'big_green_btw_rounds_box': 3, 'challenge_btn_orange': 4, 'challenge_high_score_board': 5, 'curr_state': 6, 'did_you_enjoy_this_location': 7, 'final_scores_box_beige': 8, 'finished_legs_box': 9, 'game_about_to_start_box_white': 10, 'game_finished_well_done_big_box': 11, 'game_finished_white_box': 12, 'game_title': 13, 'guess': 14, 'guess_grey': 15, 'guess_w_icon_only': 16, 'high_score_box': 17, 'in_game_mini_map': 18, 'invite_friends': 19, 'leader_board': 20, 'left_menu_dark': 21, 'loading_loc_white': 22, 'make_a_guess': 23, 'next_orange_btn': 24, 'next_round': 25, 'other': 26, 'participants_box': 27, 'play': 28, 'play_again': 29, 'play_current_leg': 30, 'play_next_round': 31, 'points_bar': 32, 'points_bar_two_bars': 33, 'refresh_btn': 34, 'setup_round_time_limit_box': 35, 'share_challenge_box_white': 36, 'show_full_results': 37, 'show_high_score': 38, 'start_challenge_orange

In [9]:
model_name = "torchvision"
model, model_type = get_model(geo_screens.parser, backend_type=model_name)
geo_screens.set_model_type(model_type)
light_model = build_module(model_name, model, learning_rate=1e-4).cuda(DEVICE)
light_model.load_state_dict(
    torch.load(
        "/home/gbiamby/proj/geo/scripts/output/gs005_model_torchvision-lr_0.0001-ratios_0.08_to_2.0-sizes_32_to_512-detsperimg_512/geoscreens-epoch=19-coco_ap50_COCOMetric=0.97.ckpt"
    )["state_dict"]
)

learning_ratee:  0.0001


<All keys matched successfully>

In [10]:
light_model.device

device(type='cuda', index=2)

## Run on one video

In [14]:
def image_from_url(url):
    img = PIL.Image.open(url)
    return np.array(img)


def batch(iterable, n=1):
    """Splits an iterable / list-like into batches of size n"""
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx : min(ndx + n, l)]


def segment_video(video_path: Path, output_video: bool = False):
    """
    Params:
        output_video: if True, saves output to video (untested, might not work yet)
    Returns:
        Dict: keys = frame index, value = a dict of detections that looks something like:
            {
                "label_ids": [17, 39],
                "scores": [0.5707356929779053, 0.5458141565322876],
                "bboxes": [
                    {
                        "xmin": 522.35400390625,
                        "ymin": 177.13229370117188,
                        "xmax": 640.0,
                        "ymax": 362.1326599121094,
                    },
                    {
                        "xmin": 537.4188232421875,
                        "ymin": 139.51719665527344,
                        "xmax": 635.33642578125,
                        "ymax": 157.04588317871094,
                    },
                ],
            }
    """
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("Error opening input video: {}".format(video_path))

    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"num_frames: {num_frames:,}")
    seconds = round(0, 2)
    sample_every_seconds = 0.25

    # out_ptr = None
    # if output_video:
    #     frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
    #     out_ptr = cv2.VideoWriter(
    #         output_video,
    #         cv2.VideoWriter_fourcc("m", "p", "4", "v"),
    #         25,
    #         (frame_width, frame_height),
    #     )

    infer_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(640), tfms.A.Normalize()])
    frame_counter = 0
    detections = {}
    p_bar = tqdm(total=num_frames)
    while cap.isOpened():
        # if frame_counter >= 60:
        #     break
        cap.set(cv2.CAP_PROP_POS_MSEC, (seconds * 1000))
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break

        raw_frames = [np.array(frame)]
        infer_ds = Dataset.from_images(raw_frames, infer_tfms)
        batch, samples = models.torchvision.retinanet.build_infer_batch(infer_ds)
        preds = models.torchvision.retinanet.predict(model, infer_ds, detection_threshold=0.5)
        if preds:
            assert len(preds) == 1, "Expected list of size 1."
            preds = preds[0]
            detections[frame_counter] = {
                "label_ids": [int(l) for l in preds.detection.label_ids],
                "scores": preds.detection.scores.tolist(),
                "bboxes": [
                    {
                        "xmin": float(box.xmin),
                        "ymin": float(box.ymin),
                        "xmax": float(box.xmax),
                        "ymax": float(box.ymax),
                    }
                    for box in preds.detection.bboxes
                ],
            }
        # if out_ptr:
        #     # write frame into output file.
        #     out_ptr.write(new_frame)
        # else:
        #     # show the frame online, mainly used for real-time speed test.
        #     cv2.imshow("Frame", new_frame)
        #     # Press Q on keyboard to  exit
        #     if cv2.waitKey(1) & 0xFF == ord("q"):
        #         break
        frame_counter += 1
        seconds = round(seconds + sample_every_seconds, 2)
        # print(seconds, seconds * 1000)
        p_bar.update(10)
    p_bar.close()

    print(frame_counter, seconds)
    return detections


frame_detections = segment_video(VIDEO_PATH / "178LA7LXoCs.mp4")

num_frames: 45,115


  0%|          | 0/45115 [00:00<?, ?it/s]

6022 1505.5


In [17]:
frame_detections[1]

{'label_ids': [18, 41, 15],
 'scores': [0.993131697177887, 0.9442188143730164, 0.5944535732269287],
 'bboxes': [{'xmin': 527.5426025390625,
   'ymin': 412.3382873535156,
   'xmax': 627.6483154296875,
   'ymax': 482.1278381347656},
  {'xmin': 534.7284545898438,
   'ymin': 141.13693237304688,
   'xmax': 629.7432250976562,
   'ymax': 160.68435668945312},
  {'xmin': 526.2078247070312,
   'ymin': 483.2252197265625,
   'xmax': 625.5960083007812,
   'ymax': 494.23895263671875}]}

In [16]:
pickle.dump(frame_detections, open(OUT_PATH / "detections/178LA7LXoCs.pkl", "wb"))

In [19]:
def get_categories():
    data = json.load(
        open("/home/gbiamby/proj/geo/datasets/geoscreens_001/geoscreens_001.json", "r")
    )
    return data["categories"]


cats = get_categories()
print(cats)

[{'id': 0, 'name': 'battle_royale_wait_screen'}, {'id': 1, 'name': 'between_rounds_box_white'}, {'id': 2, 'name': 'between_rounds_box_with_orange_next'}, {'id': 3, 'name': 'big_green_btw_rounds_box'}, {'id': 4, 'name': 'challenge_btn_orange'}, {'id': 5, 'name': 'challenge_high_score_board'}, {'id': 6, 'name': 'curr_state'}, {'id': 7, 'name': 'did_you_enjoy_this_location'}, {'id': 8, 'name': 'final_scores_box_beige'}, {'id': 9, 'name': 'finished_legs_box'}, {'id': 10, 'name': 'game_about_to_start_box_white'}, {'id': 11, 'name': 'game_finished_well_done_big_box'}, {'id': 12, 'name': 'game_finished_white_box'}, {'id': 13, 'name': 'game_title'}, {'id': 14, 'name': 'guess'}, {'id': 15, 'name': 'guess_grey'}, {'id': 16, 'name': 'guess_w_icon_only'}, {'id': 17, 'name': 'high_score_box'}, {'id': 18, 'name': 'in_game_mini_map'}, {'id': 19, 'name': 'invite_friends'}, {'id': 20, 'name': 'leader_board'}, {'id': 21, 'name': 'left_menu_dark'}, {'id': 22, 'name': 'loading_loc_white'}, {'id': 23, 'nam

In [61]:
def classify_frame(dets: dict):
    return [
        (label_id, cats[label_id], score, "")
        for label_id, score, bboxes in zip(dets["label_ids"], dets["scores"], dets["bboxes"])
    ]


ui_to_status = {
    tuple(): "out-of-game",
    tuple(sorted(["status_bar", "in_game_mini_map", "guess_grey"])): "in-game",
}
for i, (frame_id, frame_dets) in enumerate(frame_detections.items()):
    if i >= 120:
        break
    print("")
    print(f"{i/4.0:04} sec")
    labels = tuple(sorted([cats[l]["name"] for l in frame_dets["label_ids"]]))
    print(labels)
    # for in ui_to_status.items()
    if labels in ui_to_status:
        print("status: ", ui_to_status[labels])
    print(classify_frame(frame_dets))


00.0 sec
('guess_grey', 'in_game_mini_map', 'status_bar')
status:  in-game
[(18, {'id': 18, 'name': 'in_game_mini_map'}, 0.9932945370674133, ''), (41, {'id': 41, 'name': 'status_bar'}, 0.9429932832717896, ''), (15, {'id': 15, 'name': 'guess_grey'}, 0.5963080525398254, '')]

0.25 sec
('guess_grey', 'in_game_mini_map', 'status_bar')
status:  in-game
[(18, {'id': 18, 'name': 'in_game_mini_map'}, 0.993131697177887, ''), (41, {'id': 41, 'name': 'status_bar'}, 0.9442188143730164, ''), (15, {'id': 15, 'name': 'guess_grey'}, 0.5944535732269287, '')]

00.5 sec
('guess_grey', 'in_game_mini_map', 'status_bar')
status:  in-game
[(18, {'id': 18, 'name': 'in_game_mini_map'}, 0.9931173324584961, ''), (41, {'id': 41, 'name': 'status_bar'}, 0.9441524147987366, ''), (15, {'id': 15, 'name': 'guess_grey'}, 0.5962186455726624, '')]

0.75 sec
('guess_grey', 'in_game_mini_map', 'status_bar')
status:  in-game
[(18, {'id': 18, 'name': 'in_game_mini_map'}, 0.9931808710098267, ''), (41, {'id': 41, 'name': 'stat

In [43]:
52 / 4

13.0

In [ ]:
# def classify_frame(dets: dict):
#     return [
#         (label_id, cats[label_id], score, "")
#         for label_id, score, bboxes in zip(dets["label_ids"], dets["scores"], dets["bboxes"])
#     ]


# ui_to_status = {
#     tuple(): "out-of-game",
#     tuple(sorted(['status_bar', 'in_game_mini_map', 'guess_grey'])): "in-game",

# }
# for i, (frame_id, frame_dets) in enumerate(frame_detections.items()):
#     if i >= 120:
#         break
#     print("")
#     print(f"{i/4.0:04} sec")
#     frame_labels = tuple(sorted([cats[l]["name"] for l in frame_dets["label_ids"]]))
#     print(frame_labels)
#     for _labels, status in ui_to_status.items():
#         for l in _labels:
#             if l in frame_labels
#     if labels in ui_to_status:
#         print("status: ", ui_to_status[labels])
#     print(classify_frame(frame_dets))